Le dataset sur lequel nous travaillons est issu d'une publication de Nuno Antonio, Ana Almeida et Luis Nunes pour Data in Brief (Volume 22, Février 2019). Ce dataset agrège les données résultantes de deux sources : la première d'un hôtel de campagne et la seconde d'un hôtel situé en ville.<br>
Le lien du dataset est le suivant : https://www.kaggle.com/datasets/jessemostipak/hotel-booking-demand/data

L'annulation d'une réservation séjour touristique peut-être multifactorielle : aléas familiaux, météorologiques, politiques... <br>
Cependant, d'aucuns pourraient penser qu'il est possible, de par certaines caractéristiques des clients ou de la réservation, de prédire si oui ou non un client sera enclin à annuler une réservation. Le dataset que nous utilisons fait l'objet de nombreux travaux étudiant cette question. <br>Afin d'aller au-delà, nous avons décidé d'étudier les valeurs SHAP dans le but de répondre à la problématique suivante : **Les comportements de réservation sont-ils plus impactants que d'autres paramètres sur le maintien d'une réservation d'un hôtel ?** <br> En effet, notre premier postulat face aux variables de ce dataset est qu'un client déjà reconnu comme ayant ou non annulé une réservation dans le passé sera ou non plus enclin à annuler une prochaine réservation et qu'ainsi les variables rendant compte de ces faits ont une importance plus grande que les autres dans la classification finale du modèle. 

Notre projet se déclinera de la sorte :
- Création d'un modèle de classification binaire performant (FAIT)
- Regroupement des variables prédictives en type de variables (par exemple variables socio-démographiques, variables de type d'hôtels, variables de caractéristique du séjour... et évidemment variables de comportements de réservation) (FAIT)
- Evaluation des Mean Absolute SHAP Values de chacun de ces groupes de variables (FAIT)
- Etude statistique sur les SHAP Values pour déterminer si les différences de Mean Absolute SHAP Values éventuellement constatées sont en effet significatives (test de Student -> test de Shapiro Wilk ou Q-Q plot + test de Levene) (EN COURS)


<h1 style="color:gold;">Importations</h1>


In [1]:
#Importations des modules nécessaires

import pandas as pd
import shap
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import networkx as nx
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.base import clone
from tqdm import tqdm
import xgboost as xgb
import pandas as pd
import gc

c:\Users\gwelo\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Importation des données de notre dataset composé de 119 390 lignes et 31 variables prédictives

chemin_fichier = "https://raw.githubusercontent.com/firasbouzazi/Hotel-reservation/main/hotel_bookings.csv"
df = pd.read_csv(chemin_fichier, sep=',', encoding='utf-8')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

<h1 style="color:gold;">Traitement des données</h1>